In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')

In [4]:
data = [train_df,test_df]

In [145]:
test_df.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code
0,H11920,7,1.000000,0,V,N,O,R,300.0,1,60.0,3,3310
1,H11921,4,0.997268,0,V,N,O,R,300.0,1,60.0,3,3310
2,H9805,7,0.369863,0,V,V,V,U,790.0,1,60.0,.,3310
3,H7493,6,1.000000,0,V,N,O,R,1405.0,1,16.0,3,3321
4,H7494,4,1.000000,0,V,N,O,R,1405.0,1,16.0,3,3321


In [143]:
submission.head()

,Customer Id,Claim
0,H0,1
1,H10000,1
2,H10001,1
3,H10002,1
4,H10003,1


1.000000    5325
0.997268     547
0.000000     165
0.747945      60
0.495890      49
            ... 
0.797260       1
0.336986       1
0.372603       1
0.546448       1
0.698630       1
Name: Insured_Period, Length: 401, dtype: int64

In [108]:
dummable = ['Building_Type','Residential', 'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement','NumberOfWindows']

In [49]:
train_df['NumberOfWindows'].value_counts()

   .    3551
4        939
3        844
5        639
2        363
6        306
7        211
8        116
1         75
>=10      67
9         49
Name: NumberOfWindows, dtype: int64

In [45]:
#Fill with mean or drop for train_df
for datum in data:v  
    x = round(datum['Building Dimension'].mean())
    datum['Building Dimension'].fillna(x,inplace=True)

In [39]:
train_df['Date_of_Occupancy'].value_counts() #Fill with mean,mode and median in varying proportions


1960.0    1818
1980.0    1155
1988.0     530
1900.0     371
1970.0     285
          ... 
1908.0       1
1946.0       1
1939.0       1
1931.0       1
1927.0       1
Name: Date_of_Occupancy, Length: 134, dtype: int64

In [44]:
for datum in data:
    x = datum['Geo_Code'].mode()[0]
    datum['Geo_Code'].fillna(x,inplace=True) #Fill with mode.

In [42]:
def replace_nan(df,feature_name,new_values,weights):
    '''
    A function that replaces the nan values with some different values in different proportions.
    
    Parameters:
    --------------------------------------------------
        df: a dataframe  
        feature_name: the name of the feature to be worked on as a string
        new_values: this is a list of the values to replace the nan values as strings
        weights: this is a list of the weights assigned to each value. Its sum is to be equal to 1.
    Returns:
    ----------------------------------------------------------------------
        The edited feature as a numpy array.
    '''
    assert sum(weights) ==1.0,'Weights do not add up to one...'
    assert len(new_values)==len(weights),'New values do not correspond with weights'
    from random import choices
    mask= df[feature_name].isna()
    length = sum(mask)
    replacement = choices(new_values,weights =weights,k=length)
    df.loc[mask,feature_name]=replacement
    return df[feature_name]

In [43]:
for datum in data:
    y = round(datum['Date_of_Occupancy'].mean())
    z = datum['Date_of_Occupancy'].mode()[0]
    datum['Date_of_Occupancy'] = replace_nan(datum,'Date_of_Occupancy',[y,z],[0.5,0.5])

In [52]:
train_df.dtypes

Customer Id            object
YearOfObservation       int64
Insured_Period         object
Residential            object
Building_Painted       object
Building_Fenced        object
Garden                 object
Settlement             object
Building Dimension    float64
Building_Type          object
Date_of_Occupancy     float64
NumberOfWindows        object
Geo_Code               object
Claim                   int64
dtype: object

In [51]:
for datum in data:
    datum['YearOfObservation']= 2020 - datum['YearOfObservation']
    datum['Date_of_Occupancy']= 2020 - datum['Date_of_Occupancy']
    datum['Insured_Period']=datum['Insured_Period'].astype('float64')
    datum['Residential']=datum['Residential'].astype('object')
    datum['Building_Type']=datum['Building_Type'].astype('object')
    datum['Geo_Code']=datum['Geo_Code'].astype('int64')

In [109]:
dum_train = pd.get_dummies(train_df[dummable])
dum_train['Customer Id']=train_df['Customer Id']
train_data = pd.merge(train_df,dum_train,on='Customer Id')
train_data.drop(dummable,axis=1,inplace=True)

In [110]:
dum_test = pd.get_dummies(test_df[dummable])
dum_test['Customer Id']=test_df['Customer Id']
test_data = pd.merge(test_df,dum_test,on='Customer Id')
test_data.drop(dummable,axis=1,inplace=True)

In [113]:
len(train_data.columns)==len(test_data.columns)+1

True

In [114]:
Y_train = train_df['Claim']
X_train = train_data.drop(['Customer Id','Claim','Geo_Code'],axis=1)
X_test = test_data.drop(['Customer Id','Geo_Code'],axis=1)

In [92]:
from sklearn.ensemble import RandomForestClassifier

In [115]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

99.2

In [148]:
submission = pd.DataFrame()
submission['Customer Id'] = test_df['Customer Id']
submission['Claim'] = Y_prediction
submission.to_csv('solution00.csv',index=False )

In [64]:
from xgboost import XGBClassifier

In [149]:
model=XGBClassifier()
model.fit(X_train,Y_train)
Y_xgb = model.predict(X_test)

In [135]:
X_test.tail()

,YearOfObservation,Insured_Period,Building Dimension,Date_of_Occupancy,Building_Type_1,Building_Type_2,Building_Type_3,Building_Type_4,Residential_0,Residential_1,...,NumberOfWindows_1,NumberOfWindows_2,NumberOfWindows_3,NumberOfWindows_4,NumberOfWindows_5,NumberOfWindows_6,NumberOfWindows_7,NumberOfWindows_8,NumberOfWindows_9,NumberOfWindows_>=10
3064,5,1.0,1667.0,120.0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3065,8,1.0,1667.0,72.0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3066,8,1.0,1667.0,27.0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3067,7,1.0,1667.0,220.0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3068,8,1.0,1667.0,70.0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
submission.tail()

,Customer Id,Claim
3063,H9987,1
3064,H9988,1
3065,H9994,1
3066,H9996,1
3067,H9998,1


In [150]:
submission1 = pd.DataFrame()
submission1['Customer Id'] = test_df['Customer Id']
submission1['Claim'] = Y_xgb
submission1.to_csv('solution0a.csv',index=False )